In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from string import ascii_letters,ascii_lowercase,ascii_uppercase
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from pathlib import Path
import category_encoders as ce
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import os
print(os.listdir('/kaggle/input/cat-in-the-dat'))
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
input_folder = Path('/kaggle/input/cat-in-the-dat')

In [ ]:
train_df = pd.read_csv(input_folder/"train.csv")

In [ ]:
test_df = pd.read_csv(input_folder/'test.csv')

In [ ]:
X = train_df[train_df.columns.difference(['id','target'])]

In [ ]:
y = train_df['target']

In [ ]:
train_df_no_id = train_df[train_df.columns.difference(['id'])]

In [ ]:
train_df_no_id.columns

In [ ]:
# f,ax = plt.subplots(3,2,figsize=(15,15))
# ax = ax.flatten()
# cols = ["bin_{}".format(i) for i in range(5)]
# for i,axi in enumerate(ax[:-1]):
#     sns.catplot(y=cols[i], hue="target", kind="count",
#             palette="pastel", edgecolor=".6",
#             data=train_df_no_id,ax=axi);

In [ ]:
train_df_no_id.day.value_counts()

In [ ]:
train_df_no_id.groupby('month')['target'].value_counts()

In [ ]:
# f,ax = plt.subplots(1,2,figsize=(15,15))
# ax = ax.flatten()
# cols = ['month','day']
# for i,axi in enumerate(ax):
#     sns.catplot(y=cols[i], hue="target", kind="count",
#             palette="pastel", edgecolor=".6",
#             data=train_df_no_id,ax=axi);

In [ ]:
for i in range(10):
    colname = "nom_{}".format(i)
    print(colname,len(train_df_no_id[colname].unique()))

In [ ]:
for i in range(6):
    colname = "ord_{}".format(i)
    print(colname,len(train_df_no_id[colname].unique()),train_df_no_id[colname].unique())

In [ ]:
ord_5_unique_sorted  = train_df_no_id.ord_5.unique()
ord_5_unique_sorted.sort()
print(ord_5_unique_sorted)

There are categories which are not in the training set

In [ ]:
unique_cols = {}
for col in train_df_no_id.columns.difference(['target']):
    unique_cols[col] = set(test_df[col].unique()) - set(train_df_no_id[col].unique())

In [ ]:
unique_cols

In [ ]:
print(train_df_no_id.isnull().sum())
print(test_df.isnull().sum())

In [ ]:
bin_encoder = ce.binary.BinaryEncoder(verbose = 1,cols = ['bin_{}'.format(i) for i in range(5)],return_df=True)

In [ ]:
bin_encoder.fit(X,y)

In [ ]:
X_encoded_bin = bin_encoder.transform(X)

In [ ]:
X_encoded_bin

ord_0 3 [2 1 3]

ord_1 5 ['Grandmaster' 'Expert' 'Novice' 'Contributor' 'Master']

ord_2 6 ['Cold' 'Hot' 'Lava Hot' 'Boiling Hot' 'Freezing' 'Warm']

ord_3 15 ['h' 'a' 'i' 'j' 'g' 'e' 'd' 'b' 'k' 'f' 'l' 'n' 'o' 'c' 'm']

ord_4 26 ['D' 'A' 'R' 'E' 'P' 'K' 'V' 'Q' 'Z' 'L' 'F' 'T' 'U' 'S' 'Y' 'B' 'H' 'J'
 'N' 'G' 'W' 'I' 'O' 'C' 'X' 'M']

In [ ]:
ascii_letters,ascii_lowercase

In [ ]:
X_encoded_bin.ord_5.str[1].unique()

In [ ]:
maps = []

ord_0_dict = {'col':'ord_0'}
ord_0_dict['mapping'] = {1:1,2:2,3:3}
maps.append(ord_0_dict)

ord_1_dict = {'col' : 'ord_1'}
ord_1_dict['mapping'] = {'Grandmaster':4, 'Expert':2, 'Novice':0, 'Contributor':1, 'Master':3}
maps.append(ord_1_dict)

ord_2_dict = {'col' : 'ord_2'}
ord_2_dict['mapping'] = {'Cold':1, 'Hot':3 ,'Lava Hot':5, 'Boiling Hot':4 ,'Freezing':0 ,'Warm':2}
maps.append(ord_2_dict)

ord_3_dict = {'col' : 'ord_3'}
ord_3_dict['mapping'] = { x:i for i,x in enumerate(ascii_lowercase)}
maps.append(ord_3_dict)

ord_4_dict = {'col' : 'ord_4'}
ord_4_dict['mapping'] = { x:i for i,x in enumerate(ascii_uppercase)}
maps.append(ord_4_dict)

ord_5_dict = {'col' : 'ord_5'}
ord_5_dict['mapping'] = { x:i for i,x in enumerate(ord_5_unique_sorted)}
maps.append(ord_5_dict)

In [ ]:
maps

In [ ]:
ord_encoder = ce.ordinal.OrdinalEncoder(verbose = 1, cols = ['ord_{}'.format(i) for i in range(5)], return_df = True, mapping = maps)

In [ ]:
ord_encoder.fit(X_encoded_bin,y)

In [ ]:
X_encoded_bin_ord = ord_encoder.transform(X_encoded_bin)

In [ ]:
X_encoded_bin_ord

In [ ]:
req = ['nom','ord']
for col in X_encoded_bin_ord.columns:
    if(any([x in col for x in req])):
        print(col,X_encoded_bin_ord[col].nunique())

In [ ]:
one_hot_cols = ["nom_{}".format(i) for i in range(6)] + ["ord_{}".format(i) for i in range(6)]
print(one_hot_cols)

In [ ]:
ohe = ce.one_hot.OneHotEncoder(cols=one_hot_cols,return_df=True)

In [ ]:
ohe.fit(X_encoded_bin_ord,y)

In [ ]:
X_all_encoded_hot = ohe.transform(X_encoded_bin_ord)

In [ ]:
X_all_encoded_hot.shape

In [ ]:
hash_encoder = ce.hashing.HashingEncoder(verbose = 1, return_df = True, cols = ["nom_{}".format(i) for i in range(5,10)],max_process=8,max_sample=10000)

In [ ]:
hash_encoder.fit(X_all_encoded_hot,y)

In [ ]:
X_all_encoded = hash_encoder.transform(X_all_encoded_hot)

In [ ]:
cyclic_cols = ['day','month']
transforms = {'sin':np.sin,'cos':np.cos}
max_cols = [np.max(X_all_encoded['day']),np.max(X_all_encoded['month'])]

In [ ]:
for i,col in enumerate(cyclic_cols):
    for t,f in transforms.items():
        X_all_encoded[col+t] = f(X_all_encoded[col]*2*np.pi/max_cols[i])
        

In [ ]:
X_all_encoded.drop(labels=['day','month'],inplace=True,axis=1)

In [ ]:
y.value_counts()

In [ ]:
scale_pos_weight = 208236/91764

In [ ]:
scale_pos_weight

In [ ]:
classifier = xgb.XGBClassifier(max_depth = 15, n_estimators=5000, scale_pos_weight=2.27,
                      random_state=0,tree_method='gpu_hist')

In [ ]:
classifier.fit(X_all_encoded,y)

In [ ]:
classifier.feature_importances_.shape

In [ ]:
# parameters = dict(max_depth=[10], learning_rate=[1e-2], n_estimators=[200])
# parameters = dict(subsample=[0.1,0.2,0.5,1.0])

In [ ]:
# clf = GridSearchCV(classifier, parameters, cv=2)

In [ ]:
# clf.fit(X_all_encoded,y)

In [ ]:
# clf.best_params_

In [ ]:
# clf.best_score_

In [ ]:
# clf.cv_results_

In [ ]:
X_test = test_df[test_df.columns.difference(['id'])]

In [ ]:
# Apply the transforms on test set

In [ ]:
test_df_encoded_bin = bin_encoder.transform(X_test)

In [ ]:
test_df_encoded_bin_ord = ord_encoder.transform(test_df_encoded_bin)

In [ ]:
import gc
gc.collect()

In [ ]:
test_df_encoded_bin_ord_hot = ohe.transform(test_df_encoded_bin_ord)

In [ ]:
test_df_encoded_bin_ord_hot.shape

In [ ]:
X_test_all_encoded = hash_encoder.transform(test_df_encoded_bin_ord_hot)

In [ ]:
X_test_all_encoded.columns

In [ ]:
for i,col in enumerate(cyclic_cols):
    for t,f in transforms.items():
        X_test_all_encoded[col+t] = f(X_test_all_encoded[col]*2*np.pi/max_cols[i])

In [ ]:
X_test_all_encoded.drop(labels=['day','month'],inplace=True,axis=1)

In [ ]:
X_test_all_encoded.shape,X_all_encoded.shape

In [ ]:
ypred = classifier.predict_proba(X_test_all_encoded)

In [ ]:
ypred[:10]

In [ ]:
np.unique(ypred,return_counts=True)

In [ ]:
sub_df = pd.DataFrame({'id':test_df.id,'target':ypred[:,1]})

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv("Download.csv",index=False)

<a href="Download.csv">Download</a>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier(n_estimators = 300,min_samples_split = 5, max_leaf_nodes = 100, n_jobs=-1)

In [ ]:
classifier.fit(X_all_encoded,y)

In [ ]:
ypred = classifier.predict(X_test_all_encoded)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score

In [ ]:
model = LogisticRegressionCV(Cs=[1e-2,1e-1,1,1e1,1e2],cv = 3, n_jobs = -1, random_state = 42)

In [ ]:
model.fit(X_all_encoded,y)

In [ ]:
model.scores_

In [ ]:
ypred = model.predict(X_test_all_encoded)